# Case study 1:

In [40]:
# Imports
import pandas as pd
import numpy as np
import re
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from IPython.core.display import display, HTML
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')
# Increase the width of the notebook for displaying DataFrames
display(HTML("<style>.container { width:75% !important; }</style>"))

C:\Users\Guang\AppData\Local\Temp\ipykernel_14608\478144586.py:8: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Guang\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Guang\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [41]:
data = pd.read_csv("preprocessed_data.csv",low_memory=False)
data.head()

,Id,Title,Company,Date,Location,Area,Classification,SubClassification,Requirement,FullDescription,LowestSalary,HighestSalary,JobType,AverageSalary
0,37404348,Casual Stock Replenisher,Aldi Stores,2018-10-07,Sydney,North West & Hills District,Retail & Consumer Products,Retail Assistants,Our Casual Stock Replenishers pride themselves...,NaN,0,30,NaN,15.0
1,37404337,Casual Stock Replenisher,Aldi Stores,2018-10-07,Richmond & Hawkesbury,NaN,Retail & Consumer Products,Retail Assistants,Our Casual Stock Replenishers pride themselves...,NaN,0,30,NaN,15.0
2,37404356,RETAIL SALES SUPERSTARS and STYLISTS Wanted - ...,LB Creative Pty Ltd,2018-10-07,Brisbane,CBD & Inner Suburbs,Retail & Consumer Products,Retail Assistants,BRAND NEW FLAGSHIP STORE OPENING - SUNSHINE PLAZA,NaN,0,30,NaN,15.0
3,37404330,Team member - Belrose,Anaconda Group Pty Ltd,2018-10-07,Gosford & Central Coast,NaN,Retail & Consumer Products,Retail Assistants,Bring it on - do you love the great outdoors a...,NaN,0,30,NaN,15.0
4,37404308,"Business Banking Contact Centre Specialist, Ni...",Commonwealth Bank - Business & Private Banking,2018-10-07,Sydney,Ryde & Macquarie Park,Call Centre & Customer Service,Sales - Inbound,"We are seeking highly articulate, enthusiastic...",NaN,0,30,NaN,15.0


In [42]:
# Filter the data for computer science and IT-related jobs
filtered_data = data[data['Classification'].str.contains('Information', na=False)]

# Combine the 'Requirement' and 'FullDescription' columns
filtered_data['combined_text'] = filtered_data['Requirement'] + ' ' + filtered_data['FullDescription']

# Preprocess the text data
def preprocess_text(text):
    # Tokenize and convert to lowercase
    tokens = nltk.word_tokenize(text.lower())
    
    # Remove non-alphabetic characters
    words = [word for word in tokens if word.isalpha()]
    
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    
    # Lemmatize the words
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    
    return ' '.join(words)

filtered_data['processed_text'] = filtered_data['combined_text'].apply(lambda x: preprocess_text(str(x)))

# Vectorize the text data
vectorizer = CountVectorizer(stop_words='english', max_df=0.95, min_df=2)
text_matrix = vectorizer.fit_transform(filtered_data['processed_text'])

# Apply topic modeling using LDA
lda = LatentDirichletAllocation(n_components=30, random_state=42)
lda.fit(text_matrix)

# Print the top 10 words for each topic
for index, topic in enumerate(lda.components_):
    print(f'Top 10 words for Topic #{index+1}:')
    print([vectorizer.get_feature_names_out()[i] for i in topic.argsort()[-10:]])
    print('\n')

C:\Users\Guang\AppData\Local\Temp\ipykernel_14608\2101394758.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['combined_text'] = filtered_data['Requirement'] + ' ' + filtered_data['FullDescription']
C:\Users\Guang\AppData\Local\Temp\ipykernel_14608\2101394758.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['processed_text'] = filtered_data['combined_text'].apply(lambda x: preprocess_text(str(x)))


Top 10 words for Topic #1:
['management', 'team', 'role', 'business', 'skill', 'project', 'experience', 'strong', 'nbsp', 'br']


Top 10 words for Topic #2:
['learning', 'nbsp', 'data', 'technology', 'work', 'people', 'experience', 'strong', 'team', 'li']


Top 10 words for Topic #3:
['role', 'solution', 'service', 'work', 'business', 'skill', 'experience', 'li', 'strong', 'nbsp']


Top 10 words for Topic #4:
['window', 'role', 'server', 'ul', 'service', 'network', 'experience', 'support', 'strong', 'li']


Top 10 words for Topic #5:
['ul', 'process', 'technical', 'design', 'strong', 'requirement', 'experience', 'solution', 'business', 'li']


Top 10 words for Topic #6:
['tool', 'ul', 'quality', 'integration', 'automation', 'strong', 'experience', 'testing', 'li', 'test']


Top 10 words for Topic #7:
['day', 'note', 'recruitment', 'send', 'today', 'resume', 'water', 'apply', 'button', 'clicking']


Top 10 words for Topic #8:
['role', 'credit', 'institution', 'strong', 'transport', 'inv

My codes 
1. Filters Data for IT-related jobs
2. Preprocesses the text data(The process function tokenizes the input text, removes non-alphabetic characters, removes stop words using the NLTK library, and lemmatizes the words using the WordNetLemmatizer from NLTK. )
3. Applies topic modeling using Latent Dirichlet Allocation (LDA) to identify the top skills and subjects mentioned in the job listings.

The above result shows that the most important skill from job market is that:"cyber security", "web", "sql", "ios", "machine learning" and "programing language"

# Case study 2

In [43]:
# Import
import pandas as pd
import re
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Guang\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Guang\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [44]:
data = pd.read_csv("preprocessed_data.csv",low_memory=False)

In [46]:
# Combine the 'Requirement' and 'FullDescription' columns
data['combined_text'] = data['Requirement'] + ' ' + data['FullDescription']

# Preprocess the text data
def preprocess_text(text):
    # Tokenize and convert to lowercase
    tokens = nltk.word_tokenize(text.lower())
    
    # Remove non-alphabetic characters
    words = [word for word in tokens if word.isalpha()]
    
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    
    # Lemmatize the words
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    
    return ' '.join(words)

data['processed_text'] = data['combined_text'].apply(lambda x: preprocess_text(str(x)))

# Edit Candidate's profile here
candidate_profile = '''
Mathew is a student in computer science. He has skills like machine learning and cyber security
'''

# Preprocess the candidate's profile
processed_candidate_profile = preprocess_text(candidate_profile)

# Vectorize the text data using TF-IDF
vectorizer = TfidfVectorizer(stop_words='english')
text_matrix = vectorizer.fit_transform(data['processed_text'])
candidate_vector = vectorizer.transform([processed_candidate_profile])

# Compute cosine similarity
similarity_scores = cosine_similarity(candidate_vector, text_matrix)

# Get the top 10 job indices and their similarity scores
top_10_job_indices = np.argsort(similarity_scores[0])[-10:][::-1]
top_10_job_scores = np.sort(similarity_scores[0])[-10:][::-1]

In [47]:
# Print the top 10 job recommendations
print("Top 10 job recommendations for the candidate:")
for i, job_index in enumerate(top_10_job_indices):
    print(f"{i+1}. Job ID: {data.loc[job_index, 'Id']}, Similarity Score: {top_10_job_scores[i]}")

Top 10 job recommendations for the candidate:
1. Job ID: 37408864, Similarity Score: 0.42134648229395555
2. Job ID: 37455360, Similarity Score: 0.4207103915603747
3. Job ID: 37490442, Similarity Score: 0.41001996845535865
4. Job ID: 37597983, Similarity Score: 0.4093404192805201
5. Job ID: 37997807, Similarity Score: 0.40167285400696134
6. Job ID: 37554941, Similarity Score: 0.3814574296069183
7. Job ID: 37515146, Similarity Score: 0.3813083888849317
8. Job ID: 37994290, Similarity Score: 0.38122283387662564
9. Job ID: 37954799, Similarity Score: 0.38122283387662564
10. Job ID: 38064961, Similarity Score: 0.3783559598060111


1. Preprocess the job market dataset and the candidate's profile (The process function tokenizes the input text, removes non-alphabetic characters, removes stop words using the NLTK library, and lemmatizes the words using the WordNetLemmatizer from NLTK. ).
2. Vectorize the text data using TF-IDF, which converts the text into a numerical representation.
3. Compute the cosine similarity between the candidate's profile vector and each job listing vector.
4. Sort the job listings based on their similarity scores and recommend the top 10 jobs with the highest similarity scores.